# Reconocimiento de dígitos con KNN

Veamos como interactuar con las clases implementadas en el módulo de C++ haciendo una experimentación muy simplificada, y a modo de ejemplo, del método KNN para reconocer digitos

## Compilando el módulo desde C++

Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/rociodcasco/miniconda3/envs/tp2/bin/python3 (found version "3.6.5") 
-- Found PythonLibs: /home/rociodcasco/miniconda3/envs/tp2/lib/libpython3.6m.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/mnt/c/Users/Rocío/Documents/GitHub/metnum

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import metnum

/mnt/c/Users/Rocío/Documents/GitHub/metnum_tp2/notebooks
Python 3.6.5 :: Anaconda, Inc.


## Cargando los datos

In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

data_frame = pd.read_csv("../data/train.csv")
X = data_frame[data_frame.columns[1:]].values
y = data_frame["label"].values.reshape(-1, 1)

Es un csv, con columnas por cada pixel

### K-fold

In [4]:
%%time

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

exactitudes = []

alfa_componentes_principales = 40
k_vecinos_mas_cercanos = 5

for K in range(3,5+1) :
    exactitudes.append([])
    tamanio_particiones = int(X.shape[0]/K)
    for i in range(0,K) :
        limite_bajo = tamanio_particiones*i
        limite_alto = tamanio_particiones*(i+1)
        X_entrenamiento = np.concatenate((X[:limite_bajo], X[limite_alto:]))
        y_entrenamiento = np.concatenate((y[:limite_bajo],y[limite_alto:]))
        X_validacion = X[limite_bajo:limite_alto]
        y_validacion = y[limite_bajo:limite_alto]
        
        PCA = metnum.PCA(alfa_componentes_principales)
        PCA.fit(X_entrenamiento)
        clasificador = metnum.KNNClassifier(k_vecinos_mas_cercanos)
        X_entrenamiento_PCA = PCA.transform(X_entrenamiento)
        X_validacion_PCA = PCA.transform(X_validacion)
        
        clasificador.fit(X_entrenamiento_PCA, y_entrenamiento)
        predicciones = clasificador.predict(X_validacion_PCA)
        
        exactitudes[-1].append(accuracy_score(predicciones, y_validacion))
        print("K:",K,",i:",i,"→ exactitud:",exactitudes[-1][-1])
    print("K:",K,"→ promedio:",np.mean(exactitudes[-1]),"mínimo:",np.min(exactitudes[-1]))

K: 3 ,i: 0 → exactitud: 0.9555
K: 3 ,i: 1 → exactitud: 0.9529285714285715
K: 3 ,i: 2 → exactitud: 0.9571428571428572
K: 3 → promedio: 0.9551904761904763 mínimo: 0.9529285714285715
K: 4 ,i: 0 → exactitud: 0.9552380952380952
K: 4 ,i: 1 → exactitud: 0.9577142857142857
K: 4 ,i: 2 → exactitud: 0.9528571428571428
K: 4 ,i: 3 → exactitud: 0.962
K: 4 → promedio: 0.956952380952381 mínimo: 0.9528571428571428
K: 5 ,i: 0 → exactitud: 0.9610714285714286
K: 5 ,i: 1 → exactitud: 0.9572619047619048
K: 5 ,i: 2 → exactitud: 0.9577380952380953
K: 5 ,i: 3 → exactitud: 0.9554761904761905
K: 5 ,i: 4 → exactitud: 0.9551190476190476
K: 5 → promedio: 0.9573333333333334 mínimo: 0.9551190476190476
CPU times: user 3min 8s, sys: 11.2 s, total: 3min 20s
Wall time: 3min 22s
